In [1]:
import openpathsampling as paths

# Simplest Possible OPS Example

## Set up the system

### Prepare MD engine

In [5]:
# always the most complicated bit
import openpathsampling.engines.toy as toys
import numpy as np

pes = (toys.OuterWalls([1.0,1.0], [0.0,0.0]) + 
       toys.Gaussian(-0.7, [12.0, 0.5], [-0.5, 0.0]) +
       toys.Gaussian(-0.7, [12.0, 0.5], [0.5, 0.0]))
topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)
engine = toys.Engine({'integ': toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5),
                              'n_frames_max': 5000,
                              'n_steps_per_frame': 10}, topology)
template = toys.Snapshot(coordinates=np.array([[0.0, 0.0]]),
                         velocities=np.array([[0.0, 0.0]]),
                        engine=engine)


### Define states

In [6]:
# states are volumes in a CV space: define the CV
def xval(snapshot):
    return snapshot.xyz[0][0]

cv = paths.FunctionCV("xval", xval)

In [7]:
stateA = paths.CVDefinedVolume(cv, float("-inf"), -0.5).named("A")
stateB = paths.CVDefinedVolume(cv, 0.5, float("inf")).named("B")

## Prepare path sampling

### Set up the `network` and `move_scheme`

In [8]:
network = paths.TPSNetwork(stateA, stateB)

In [9]:
scheme = paths.OneWayShootingMoveScheme(network, engine=engine)

### Prepare initial conditions

In [11]:
# I'll fake an initial trajectory
trajectory = paths.Trajectory([
    toys.Snapshot(coordinates=np.array([[-.55+k*0.1, 0.0]]),
                  velocities=np.array([[0.1, 0.0]]),
                  engine=engine)
    for k in range(12)
])

In [12]:
initial_conditions = scheme.initial_conditions_from_trajectories(trajectory)

# finished generating: still missing 0 samples
1


## Run path sampling

### Create storage and simulation object

In [13]:
storage = paths.Storage("simple.nc", "w", template=template)
simulation = paths.PathSampling(storage, scheme, initial_conditions)

### Run

In [14]:
simulation.run(1000)

Working on Monte Carlo cycle number 1000
Running for 328 seconds -  3.04 steps per second
Expected time to finish: 0 seconds
DONE! Completed 1000 Monte Carlo cycles.


## Analyze results

In [15]:
from collections import Counter

In [17]:
q = Counter()
for ch in storage.movechanges:
    q.update([ch.__class__])

In [18]:
q

Counter({openpathsampling.movechange.EmptyMoveChange: 1,
         openpathsampling.movechange.AcceptedSampleMoveChange: 578,
         openpathsampling.movechange.RejectedSampleMoveChange: 422,
         openpathsampling.movechange.RandomChoiceMoveChange: 3000,
         openpathsampling.movechange.PathSimulatorMoveChange: 1000})

In [11]:
# 1. path tree

In [12]:
# 2. committor based on shooting points

In [13]:
# 3. scheme.move_summary(storage)

In [14]:
# 4. path length histogram?